In [87]:
import math
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import kerastuner as kt
from kerastuner import HyperModel

NUMERIC_FEATURES = [
    'p1_height',
    'p2_height',
    'p1_age',
    'p2_age',
    'p1_rating',
    'p2_rating',
    'p1_dev',
    'p2_dev',
    'p1_surface_rating',
    'p2_surface_rating',
    'p1_surface_dev',
    'p2_surface_dev',
    'p1_w',
    'p2_w',
    'p1_l',
    'p2_l',
    'p1_surface_w',
    'p2_surface_w',
    'p1_surface_l',
    'p2_surface_l',
    'p1_inactive_days',
    'p2_inactive_days',
    'p1_recent_rating',
    'p2_recent_rating'
]
CATEGORICAL_FEATURES = [
    'is_hard',
    'is_clay',
    'is_grass',
    'is_bo5',
    'p1_lefty',
    'p2_lefty',
    'p1_home',
    'p2_home'
    
]

dataframe = pd.read_csv('../data/matches.csv')[:76425]
validation = pd.read_csv('../data/matches.csv')[76425:]


def build_model(preprocessing_head, inputs):
  body = tf.keras.Sequential([
    keras.layers.InputLayer(input_shape=(32,)),
    keras.layers.Dense(106, activation='relu'),
    keras.layers.Dropout(0.1285),
    keras.layers.Dense(1, activation='sigmoid')
  ])

  preprocessed_inputs = preprocessing_head(inputs)
  result = body(preprocessed_inputs)
  model = tf.keras.Model(inputs, result)

  model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                optimizer=tf.keras.optimizers.Adam(0.0021401977))
  return model


In [88]:
class MyHyperModel(HyperModel):
    def __init__(self, preprocessing_head, inputs):
        self.preprocessing_head = preprocessing_head
        self.inputs = inputs
    def build(self, hp):
        body = tf.keras.Sequential([
            keras.layers.InputLayer(input_shape=(32,)),
            keras.layers.Dense(hp.Int('hidden_size', 60, 400), activation='relu'),
            keras.layers.Dropout(hp.Float('dropout', 0, 0.5)),
            keras.layers.Dense(1, activation='sigmoid')
        ])

        preprocessed_inputs = self.preprocessing_head(self.inputs)
        result = body(preprocessed_inputs)
        model = tf.keras.Model(self.inputs, result)

        model.compile(
                      loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                      optimizer=tf.keras.optimizers.Adam(hp.Float('learning_rate', 0.0008, 0.01))
                      )
        return model
    
    def fit(self, hp, model, *args, **kwargs):
        return model.fit(
            *args,
            batch_size=hp.Int("batch_size", 2000, 6000),
            **kwargs,
        )

In [89]:
def get_model(dataframe):
    labels = dataframe.pop('p1_win')
    dataframe.pop('match_hash')
    dataframe.pop('tourney_name')
    dataframe.pop('tourney_date')
    dataframe.pop('p1_name')
    dataframe.pop('p2_name')

    dataframe_features = dataframe.copy()

    inputs = {}

    # match column names with input objects
    for name, column in dataframe_features.items():
        dtype = tf.float32
        inputs[name] = tf.keras.Input(shape=(1,), name=name, dtype=dtype)

    numeric_inputs = {name:input for name,input in inputs.items()
                    if name in NUMERIC_FEATURES}

    # normalize the numeric inputs and gather them in an array
    x = keras.layers.Concatenate()(list(numeric_inputs.values()))
    norm = keras.layers.Normalization()
    norm.adapt(np.array(dataframe[numeric_inputs.keys()]))
    all_numeric_inputs = norm(x)

    preprocessed_inputs = [all_numeric_inputs]

    for name, input in inputs.items():
        if name in NUMERIC_FEATURES:
            continue
        preprocessed_inputs.append(input)

    preprocessed_inputs_cat = keras.layers.Concatenate()(preprocessed_inputs)
    preprocessing = tf.keras.Model(inputs, preprocessed_inputs_cat)

    features_dict = {name: np.array(value) for name, value in dataframe_features.items()}
    f_dict = {name:values for name, values in features_dict.items()}
    preprocessing(f_dict)

    # data_model = MyHyperModel(preprocessing, inputs)
    data_model = build_model(preprocessing, inputs)

    return data_model, features_dict, labels

In [90]:
data_model, features_dict, labels = get_model(dataframe)
_, val_features_dict, val_labels = get_model(validation)

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='best_model/best_model',
    save_weights_only=True,
    monitor='loss',
    mode='min',
    save_best_only=True)

data_model.fit(x=features_dict, y=labels, epochs=20000, batch_size=3888, 
               callbacks=[tf.keras.callbacks.EarlyStopping('loss', patience=30), 
                          model_checkpoint_callback]
               )
data_model.save('finished')

# tuner = kt.BayesianOptimization(data_model, objective='val_loss', directory='tuning', max_trials=150)
# tuner.search(x=features_dict, y=labels, epochs=20000, 
#              validation_data=(val_features_dict, val_labels),
#              callbacks=[tf.keras.callbacks.EarlyStopping('val_loss', patience=15)]
#             )
# num_rows = dataframe.shape[0]
# training_end_index = math.floor((2/3) * num_rows)
# # train_dataframe = dataframe.iloc[:training_end_index]
# # val_dataframe = dataframe.iloc[training_end_index:]
# train_dataframe = dataframe.iloc[:300]
# val_dataframe = dataframe.iloc[300:450]

Epoch 1/20000
20/20 [==============================] - 1s 20ms/step - loss: 0.6640
Epoch 2/20000
20/20 [==============================] - 0s 20ms/step - loss: 0.6291
Epoch 3/20000
20/20 [==============================] - 0s 21ms/step - loss: 0.6236
Epoch 4/20000
20/20 [==============================] - 0s 19ms/step - loss: 0.6207
Epoch 5/20000
20/20 [==============================] - 0s 20ms/step - loss: 0.6187
Epoch 6/20000
20/20 [==============================] - 0s 19ms/step - loss: 0.6176
Epoch 7/20000
20/20 [==============================] - 0s 20ms/step - loss: 0.6164
Epoch 8/20000
20/20 [==============================] - 0s 19ms/step - loss: 0.6154
Epoch 9/20000
20/20 [==============================] - 0s 22ms/step - loss: 0.6146
Epoch 10/20000
20/20 [==============================] - 0s 17ms/step - loss: 0.6138
Epoch 11/20000
20/20 [==============================] - 0s 19ms/step - loss: 0.6129
Epoch 12/20000
20/20 [==============================] - 0s 20ms/step - loss: 0.6124
E

INFO:tensorflow:Assets written to: finished\assets


INFO:tensorflow:Assets written to: finished\assets


In [114]:
# reloaded = tf.keras.models.load_model('finished')
# results = reloaded.evaluate(val_features_dict, val_labels)
# predictions = reloaded.predict(val_features_dict)

data_model.load_weights('best_model/best_model')
results = data_model.evaluate(val_features_dict, val_labels)
predictions = data_model.predict(val_features_dict)

for i in range(76425, 90000):
    pred_index = i - 76425
    print('Prediction:', predictions[pred_index], 'Result:', val_labels[i])

537/537 [==============================] - 2s 4ms/step
Prediction: [0.7814054] Result: 1
Prediction: [0.06090029] Result: 0
Prediction: [0.70027554] Result: 1
Prediction: [0.9906065] Result: 1
Prediction: [0.66310275] Result: 1
Prediction: [0.01447796] Result: 0
Prediction: [0.29734084] Result: 0
Prediction: [0.02227692] Result: 0
Prediction: [0.08424639] Result: 0
Prediction: [0.88074213] Result: 0
Prediction: [0.8235627] Result: 1
Prediction: [0.00373874] Result: 1
Prediction: [0.97688705] Result: 1
Prediction: [0.03657844] Result: 0
Prediction: [0.04180838] Result: 1
Prediction: [0.99928373] Result: 1
Prediction: [0.823021] Result: 1
Prediction: [0.99967074] Result: 1
Prediction: [0.9450415] Result: 1
Prediction: [0.34902543] Result: 1
Prediction: [0.14052829] Result: 1
Prediction: [0.18407428] Result: 0
Prediction: [0.4519601] Result: 1
Prediction: [0.37407714] Result: 0
Prediction: [0.28551745] Result: 0
Prediction: [0.00288724] Result: 0
Prediction: [0.02379187] Result: 0
Predict